# 演習問題

In [27]:
ENV["LINES"] = 10
ENV["COLUMNS"] = 1000
using DataFrames, DataFramesMeta, CSV, CategoricalArrays, Statistics, FreqTables

## 2.8.1 初期教育における少人数クラスの有効性

In [11]:
star_df = CSV.read("../../data/CAUSALITY/small-class-size/data/STAR.csv", DataFrame, missingstring=["NA"])

,race,classtype,yearssmall,hsgrad,g4math,g4reading
,Int64?,Int64,Int64,Int64?,Int64?,Int64?
1,1,3,0,missing,missing,missing
2,2,3,0,missing,706,661
3,1,3,0,1,711,750
4,2,1,4,missing,672,659
5,1,2,0,missing,missing,missing
6,1,3,0,missing,missing,missing
7,1,1,4,missing,668,657
8,1,3,0,missing,missing,missing
9,1,1,4,1,709,725


In [12]:
race_d = Dict(
    1 => "white",
    2 => "black",
    3 => "others",
    4 => "hipanic",
    5 => "others",
    6 => "others",
)
classtype_d = Dict(
    1 => "small",
    2 => "normal",
    3 => "large",
)
star_df = @chain star_df begin
    @rtransform begin
        :race = ismissing(:race) ? missing : race_d[:race]
        :kinder = ismissing(:classtype) ? missing : classtype_d[:classtype]
    end
end

,race,classtype,yearssmall,hsgrad,g4math,g4reading,kinder
,String?,Int64,Int64,Int64?,Int64?,Int64?,String
1,white,3,0,missing,missing,missing,large
2,black,3,0,missing,706,661,large
3,white,3,0,1,711,750,large
4,black,1,4,missing,672,659,small
5,white,2,0,missing,missing,missing,normal
6,white,3,0,missing,missing,missing,large
7,white,1,4,missing,668,657,small
8,white,3,0,missing,missing,missing,large
9,white,1,4,1,709,725,small


In [20]:
@rsubset(star_df, :kinder == "small") |> describe

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,Type
1,race,,black,,white,1,"Union{Missing, String}"
2,classtype,1.0,1,1.0,1,0,Int64
3,yearssmall,2.70158,1,3.0,4,0,Int64
4,hsgrad,0.83592,0,1.0,1,998,"Union{Missing, Int64}"
5,g4math,709.185,487,710.0,821,1160,"Union{Missing, Int64}"
6,g4reading,723.391,528,724.0,836,1174,"Union{Missing, Int64}"
7,kinder,,small,,small,0,String


In [21]:
@rsubset(star_df, :kinder == "normal") |> describe

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,Type
1,race,,black,,white,2,"Union{Missing, String}"
2,classtype,2.0,2,2.0,2,0,Int64
3,yearssmall,0.205561,0,0.0,3,0,Int64
4,hsgrad,0.825162,0,1.0,1,1113,"Union{Missing, Int64}"
5,g4math,709.521,487,710.0,821,1352,"Union{Missing, Int64}"
6,g4reading,719.89,528,723.0,836,1358,"Union{Missing, Int64}"
7,kinder,,normal,,normal,0,String


In [26]:
@chain star_df begin
    @rsubset(:kinder in ["small", "normal"])
    groupby(:kinder)
    @combine begin
        :math_33 = quantile(skipmissing(:g4math), [.33])
        :math_66 = quantile(skipmissing(:g4math), [.66])
        :reading_33 = quantile(skipmissing(:g4reading), [.33])
        :reading_66 = quantile(skipmissing(:g4reading), [.66])
    end
end

,kinder,math_33,math_66,reading_33,reading_66
,String,Float64,Float64,Float64,Float64
1,small,694.0,726.0,705.0,741.0
2,normal,696.0,724.0,705.0,740.0


In [28]:
prop(freqtable(star_df, :kinder, :yearssmall))

3×5 Named Matrix{Float64}
kinder ╲ yearssmall │          0           1           2           3           4
────────────────────┼───────────────────────────────────────────────────────────
large               │   0.315573    0.015336  0.00948617    0.012332         0.0
normal              │    0.31004   0.0150198  0.00916996   0.0126482         0.0
small               │        0.0   0.0910672    0.043004     0.03083    0.135494

In [29]:
@chain star_df begin
    groupby(:yearssmall)
    @combine begin
        :math_mean = mean(skipmissing(:g4math))
        :math_median = median(skipmissing(:g4math))
        :reading_mean = mean(skipmissing(:g4reading))
        :reading_median = median(skipmissing(:g4reading))
    end
end

,yearssmall,math_mean,math_median,reading_mean,reading_median
,Int64,Float64,Float64,Float64,Float64
1,0,707.979,710.0,719.875,722.0
2,1,707.552,709.0,723.147,724.5
3,2,711.914,714.0,717.868,720.0
4,3,709.617,712.0,719.899,721.0
5,4,710.052,711.0,724.665,726.0


In [30]:
@chain star_df begin
    groupby([:kinder, :yearssmall])
    @combine begin
        :math_mean = mean(skipmissing(:g4math))
        :math_median = median(skipmissing(:g4math))
        :reading_mean = mean(skipmissing(:g4reading))
        :reading_median = median(skipmissing(:g4reading))
    end
end

,kinder,yearssmall,math_mean,math_median,reading_mean,reading_median
,String,Int64,Float64,Float64,Float64,Float64
1,large,0,707.169,710.0,720.835,722.0
2,small,4,710.052,711.0,724.665,726.0
3,normal,0,708.768,710.0,718.959,722.0
4,small,1,703.0,706.0,724.667,724.5
5,normal,2,716.974,714.0,715.778,723.0
6,normal,1,706.182,696.0,726.879,730.0
7,small,2,699.143,712.0,700.571,711.0
8,large,3,705.481,710.0,714.22,720.5
9,small,3,704.5,707.0,709.148,707.0


In [37]:
@chain star_df begin
    @rsubset(:kinder in ["small", "normal"], !ismissing(:race))
    @rtransform(:isMinority = :race == "white" ? 0 : 1)
    groupby([:kinder, :isMinority])
    @combine begin
        :math_mean = mean(skipmissing(:g4math))
        :reading_mean = mean(skipmissing(:g4reading))
    end
    @orderby(:kinder, :isMinority)
end

,kinder,isMinority,math_mean,reading_mean
,String,Int64,Float64,Float64
1,normal,0,711.41,725.116
2,normal,1,698.984,690.984
3,small,0,711.19,727.839
4,small,1,698.723,700.475


In [33]:
@chain star_df begin
    @rsubset(:kinder == "normal", !ismissing(:race))
    @rtransform(:isMinority = :race == "white" ? 0 : 1)
    groupby(:isMinority)
    @combine begin
        :math_mean = mean(skipmissing(:g4math))
        :reading_mean = mean(skipmissing(:g4reading))
    end
end

,isMinority,math_mean,reading_mean
,Int64,Float64,Float64
1,0,711.41,725.116
2,1,698.984,690.984


In [38]:
@chain star_df begin
    groupby(:kinder)
    @combine begin
        :math_hsgrad = mean(skipmissing(:hsgrad))
    end
end

,kinder,math_hsgrad
,String,Float64
1,large,0.839286
2,small,0.83592
3,normal,0.825162


In [39]:
@chain star_df begin
    groupby(:yearssmall)
    @combine begin
        :math_hsgrad = mean(skipmissing(:hsgrad))
    end
end

,yearssmall,math_hsgrad
,Int64,Float64
1,0,0.828602
2,1,0.791045
3,2,0.813187
4,3,0.832461
5,4,0.877551


In [44]:
@chain star_df begin
    @rsubset(!ismissing(:race))
    @rtransform(:isMinority = :race == "white" ? 0 : 1)
    groupby([:isMinority, :kinder])
    @combine begin
        :math_hsgrad = mean(skipmissing(:hsgrad))
    end
    @orderby(:kinder, :isMinority)
end

,isMinority,kinder,math_hsgrad
,Int64,String,Float64
1,0,large,0.878748
2,1,large,0.737374
3,0,normal,0.856962
4,1,normal,0.738832
5,0,small,0.86747
6,1,small,0.747899


## 2.8.1 同性婚に関する意見の変化

In [45]:
gay_df = CSV.read("../../data/CAUSALITY/gay-marriage/data/gay.csv", DataFrame, missingstring=["NA"])

,study,treatment,wave,ssm
,Int64,String,Int64,Int64
1,1,No Contact,3,5
2,1,No Contact,4,5
3,1,No Contact,1,5
4,1,No Contact,6,5
5,1,No Contact,2,5
6,1,No Contact,7,5
7,1,No Contact,7,5
8,1,No Contact,6,4
9,1,No Contact,1,4


In [47]:
describe(gay_df, :all)

,variable,mean,std,min,q25,median,q75,max,nunique,nmissing,first,last,eltype
,Symbol,Union…,Union…,Any,Union…,Union…,Union…,Any,Union…,Int64,Any,Any,DataType
1,study,1.14923,0.356315,1,1.0,1.0,1.0,2,,0,1,2,Int64
2,treatment,,,No Contact,,,,Same-Sex Marriage Script by Straight Canvasser,5,0,No Contact,Same-Sex Marriage Script by Gay Canvasser,String
3,wave,3.75323,1.98292,1,2.0,4.0,5.0,7,,0,3,4,Int64
4,ssm,3.10672,1.62366,1,1.0,3.0,5.0,5,,0,5,5,Int64


In [48]:
@chain gay_df begin
    @rsubset(:study == 1, :wave == 1)
    groupby(:treatment)
    @combine begin
        :ssm_mean = mean(skipmissing(:ssm))
    end
end

,treatment,ssm_mean
,String,Float64
1,No Contact,3.04276
2,Recycling Script by Gay Canvasser,3.13098
3,Same-Sex Marriage Script by Gay Canvasser,3.0252
4,Recycling Script by Straight Canvasser,3.01347
5,Same-Sex Marriage Script by Straight Canvasser,3.09971


In [55]:
@chain gay_df begin
    @rsubset(:study == 1, :wave in [1, 2])
    groupby([:treatment, :wave])
    @combine begin
        :ssm_mean = mean(skipmissing(:ssm))
    end
    unstack(:treatment, :wave, :ssm_mean)
end

,treatment,1,2
,String,Float64?,Float64?
1,No Contact,3.04276,3.03961
2,Recycling Script by Gay Canvasser,3.13098,3.10745
3,Same-Sex Marriage Script by Gay Canvasser,3.0252,3.13949
4,Recycling Script by Straight Canvasser,3.01347,3.00428
5,Same-Sex Marriage Script by Straight Canvasser,3.09971,3.16186


In [56]:
@chain gay_df begin
    @rsubset(:study == 1)
    groupby([:treatment, :wave])
    @combine begin
        :ssm_mean = mean(skipmissing(:ssm))
    end
    unstack(:treatment, :wave, :ssm_mean)
end

,treatment,1,2,3,4,5,6,7
,String,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?
1,No Contact,3.04276,3.03961,3.04627,3.03595,3.17427,3.09197,3.31375
2,Recycling Script by Gay Canvasser,3.13098,3.10745,3.13854,3.13574,3.25247,3.20221,3.43194
3,Same-Sex Marriage Script by Gay Canvasser,3.0252,3.13949,3.12764,3.12857,3.32217,3.17841,3.37312
4,Recycling Script by Straight Canvasser,3.01347,3.00428,2.96402,3.02311,3.06874,3.03659,3.16597
5,Same-Sex Marriage Script by Straight Canvasser,3.09971,3.16186,3.10668,3.12314,3.27283,3.15549,3.27121


In [57]:
@chain gay_df begin
    @rsubset(:study == 2, :wave == 1)
    groupby(:treatment)
    @combine begin
        :ssm_mean = mean(skipmissing(:ssm))
    end
end

,treatment,ssm_mean
,String,Float64
1,No Contact,2.97007
2,Same-Sex Marriage Script by Gay Canvasser,2.97173


In [58]:
@chain gay_df begin
    @rsubset(:study == 2, :wave == 2)
    groupby(:treatment)
    @combine begin
        :ssm_mean = mean(skipmissing(:ssm))
    end
end

,treatment,ssm_mean
,String,Float64
1,No Contact,2.9923
2,Same-Sex Marriage Script by Gay Canvasser,3.11619


In [61]:
@chain gay_df begin
    @rsubset(:study == 2)
    groupby([:treatment, :wave])
    @combine begin
        :ssm_mean = mean(skipmissing(:ssm))
    end
    unstack(:treatment, :wave, :ssm_mean)
end

,treatment,1,2,3,4,7
,String,Float64?,Float64?,Float64?,Float64?,Float64?
1,No Contact,2.97007,2.9923,2.95166,2.96623,3.02585
2,Same-Sex Marriage Script by Gay Canvasser,2.97173,3.11619,3.10208,3.0914,3.33291


## 2.8.3 自然実験としての指導者暗殺の成功

In [63]:
leader_df = CSV.read("../../data/CAUSALITY/leader-assassination/data/leaders.csv", DataFrame, missingstring=["NA"])

,year,country,leadername,age,politybefore,polityafter,interwarbefore,interwarafter,civilwarbefore,civilwarafter,result
,Int64,String31,String31,Int64,Float64,Float64,Int64,Int64,Int64,Int64,String
1,1929,Afghanistan,Habibullah Ghazi,39,-6.0,-6.0,0,0,1,0,not wounded
2,1933,Afghanistan,Nadir Shah,53,-6.0,-7.33333,0,0,0,0,dies within a day after the attack
3,1934,Afghanistan,Hashim Khan,50,-6.0,-8.0,0,0,0,0,"survives, whether wounded unknown"
4,1924,Albania,Zogu,29,0.0,-9.0,0,0,0,0,wounded lightly
5,1931,Albania,Zogu,36,-9.0,-9.0,0,0,0,0,not wounded
6,1968,Algeria,Boumedienne,41,-9.0,-9.0,0,0,0,0,wounded lightly
7,1992,Algeria,Boudiaf,73,-2.0,-5.66667,0,0,0,1,dies within a day after the attack
8,1908,Argentina,Alcorta,48,1.0,1.0,0,0,0,0,plot stopped
9,1916,Argentina,de la Plaza,76,2.0,2.0,0,0,0,0,not wounded


In [64]:
describe(leader_df, :all)

,variable,mean,std,min,q25,median,q75,max,nunique,nmissing,first,last,eltype
,Symbol,Union…,Union…,Any,Union…,Union…,Union…,Any,Union…,Int64,Any,Any,DataType
1,year,1944.77,32.5056,1878,1920.25,1949.0,1972.0,2001,,0,1929,1934,Int64
2,country,,,Afghanistan,,,,Yugoslavia,88,0,Afghanistan,Yugoslavia,String31
3,leadername,,,Abdullah Al-Hussein,,,,del Castillo,196,0,Habibullah Ghazi,Alexander,String31
4,age,53.524,12.0398,18,45.0,52.5,61.75,81,,0,39,46,Int64
5,politybefore,-1.51867,6.44552,-10.0,-7.0,-3.0,4.0,10.0,,0,-6.0,-10.0,Float64
6,polityafter,-1.65,6.52564,-10.0,-7.0,-3.16667,3.91667,10.0,,0,-6.0,-7.66667,Float64
7,interwarbefore,0.188,0.391496,0,0.0,0.0,0.0,1,,0,0,0,Int64
8,interwarafter,0.148,0.355812,0,0.0,0.0,0.0,1,,0,0,0,Int64
9,civilwarbefore,0.216,0.41234,0,0.0,0.0,0.0,1,,0,1,0,Int64


In [66]:
@rtransform!(leader_df, :success = occursin("die", :result) ? 1 : 0)

,year,country,leadername,age,politybefore,polityafter,interwarbefore,interwarafter,civilwarbefore,civilwarafter,result,success
,Int64,String31,String31,Int64,Float64,Float64,Int64,Int64,Int64,Int64,String,Int64
1,1929,Afghanistan,Habibullah Ghazi,39,-6.0,-6.0,0,0,1,0,not wounded,0
2,1933,Afghanistan,Nadir Shah,53,-6.0,-7.33333,0,0,0,0,dies within a day after the attack,1
3,1934,Afghanistan,Hashim Khan,50,-6.0,-8.0,0,0,0,0,"survives, whether wounded unknown",0
4,1924,Albania,Zogu,29,0.0,-9.0,0,0,0,0,wounded lightly,0
5,1931,Albania,Zogu,36,-9.0,-9.0,0,0,0,0,not wounded,0
6,1968,Algeria,Boumedienne,41,-9.0,-9.0,0,0,0,0,wounded lightly,0
7,1992,Algeria,Boudiaf,73,-2.0,-5.66667,0,0,0,1,dies within a day after the attack,1
8,1908,Argentina,Alcorta,48,1.0,1.0,0,0,0,0,plot stopped,0
9,1916,Argentina,de la Plaza,76,2.0,2.0,0,0,0,0,not wounded,0


In [67]:
mean(leader_df.success)

0.216

In [71]:
@chain leader_df begin
    groupby(:success)
    @combine begin
        :polityBefore_mean = mean(:politybefore)
        :age_mean = mean(:age)
    end
end

,success,polityBefore_mean,age_mean
,Int64,Float64,Float64
1,0,-1.7432,52.7143
2,1,-0.703704,56.463


In [72]:
@chain leader_df begin
    groupby(:civilwarbefore)
    @combine begin
        :polityBefore_mean = mean(:politybefore)
        :age_mean = mean(:age)
    end
end

,civilwarbefore,polityBefore_mean,age_mean
,Int64,Float64,Float64
1,0,-1.51531,53.9592
2,1,-1.53086,51.9444


In [73]:
@chain leader_df begin
    groupby(:interwarbefore)
    @combine begin
        :polityBefore_mean = mean(:politybefore)
        :age_mean = mean(:age)
    end
end

,interwarbefore,polityBefore_mean,age_mean
,Int64,Float64,Float64
1,0,-1.66667,52.6946
2,1,-0.879433,57.1064


In [74]:
@chain leader_df begin
    groupby(:success)
    @combine begin
        :polityAfter_mean = mean(:polityafter)
        :civilwarAfter_mean = mean(:civilwarafter)
        :interwarAfter_mean = mean(:interwarafter)
    end
end

,success,polityAfter_mean,civilwarAfter_mean,interwarAfter_mean
,Int64,Float64,Float64,Float64
1,0,-1.89456,0.193878,0.168367
2,1,-0.762346,0.148148,0.0740741
